In [ ]:
import numpy as np

In [ ]:
def simulate_ising(L, beta, Nsteps):
  config=initilize_config(L)
  for _ in range(Nsteps):
    config=metropolis_step(config, beta)
  return config

In [ ]:
def initilize_config(L):
  return np.random.choice([-1,1], size=(L,L))

In [ ]:
def metropolis_step(config, beta):
  """
  One step update of metropolis algorithm

  Input
  - config: current spin configuration
  - beta: inverse temperature

  Output
  - config: updated configuration
  """
  L=config.shape[0]
  for _ in range(L*L):
    i,j = np.random.randint(0,L,size=2)
    S=config[i,j]
    # nearest neighbour spins
    S_nbr=config[(i+1)%L,j]+config[(i-1)%L,j]+config[i,(j-1)%L]+config[i,(j+1)%L]
    dE=2*S*S_nbr
    if dE<0 or np.random.rand()<np.exp(-beta*dE):
      config[i,j]*=-1
  return config

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def visualize_config(config):
  plt.imshow(config, cmap="bwr", vmin=-1, vmax=1)
  plt.colorbar(label="Spin")
  plt.title("Ising spin configuration")
  plt.axis("off")
  plt.show()

In [ ]:
visualize_config(simulate_ising(20,.1,100))

In [ ]:
def generate_dataset(L, Ts, Nsamples, Nsteps):
  """
  Generate dataset for supervised learning

  Input
  - L: lattice size
  - Ts: temperature range
  - Nsamples: size of samples for each temperature
  - Nsteps: steps of metropolitan algorithm

  Output
  - np.array(data): predictor
  - np.array(labels): target
  """
  data=[]
  labels=[]
  Tc = 2/np.log(1+np.sqrt(2)) # 2.269
  for T in Ts:
    beta=1/T
    for _ in range(Nsamples):
      config=simulate_ising(L,beta,Nsteps)
      data.append(config.copy())
      labels.append(1 if T<Tc else 0) # ordered or not
  return np.array(data), np.array(labels)

In [ ]:
L_=8
Ts_=np.linspace(1.,4.,20)
Nsamples_=100
Nsteps_=100

xs_,ys_=generate_dataset(L_,Ts_,Nsamples_,Nsteps_)

In [ ]:
visualize_config(xs_[0])
print("Label:", ys_[0])
visualize_config(xs_[-1])
print("Label:", ys_[-1])

In [ ]:
# np.savez("ising_data.npz", X=xs_, y=ys_)

In [ ]:
# data = np.load("ising_data.npz")
# xs_, ys_ = data["X"], data["y"]

In [ ]:
xs_vct_ = xs_.reshape(len(xs_),-1)

# PCA
Perform the principal component analysis and show the two-dimensional image.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA(n_components=2)
xs_pca_=pca.fit_transform(xs_vct_)

In [ ]:
def visualize_pca2(data, label):
  plt.figure(figsize=(16,9))
  scatter=plt.scatter(data[:,0],data[:,1],c=label,cmap="coolwarm",s=10)
  plt.colorbar(scatter, label="Label (1: ordered; 0: disordered)")
  plt.title("PCA of 2D Ising configuration")
  plt.xlabel("PC1")
  plt.ylabel("PC2")
  plt.grid(True)
  plt.show()

In [ ]:
visualize_pca2(xs_pca_,ys_)

# t-SNE
Perform the t-distributed Stochastic Neighbor Embedding and show the two-dimensional image.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne=TSNE(n_components=2,perplexity=30,init="pca",random_state=0)
xs_tsne_=tsne.fit_transform(xs_vct_)

In [ ]:
def visualize_tsne2(data,label):
  plt.figure(figsize=(16,9))
  scatter=plt.scatter(data[:,0],data[:,1],c=label,cmap="coolwarm",s=10)
  plt.colorbar(scatter,label="Label (1: ordered; 0: disordered)")
  plt.title("t-SNE of 2D Ising configuration")
  plt.xlabel("t-SNE 1")
  plt.ylabel("t-SNE 2")
  plt.grid(True)
  plt.show()

In [ ]:
visualize_tsne2(xs_tsne_,ys_)

# k-means
Perform the k-means clustering with $k=2$ and show the two-dimensional image.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [ ]:
kmeans=KMeans(n_clusters=2,random_state=0)
cluster_labels_=kmeans.fit_predict(xs_vct_)

In [ ]:
def visualize_tsne2_kmeans2(data,label):
  plt.figure(figsize=(16,9))
  scatter=plt.scatter(data[:,0],data[:,1],c=label,cmap="coolwarm",s=10)
  plt.colorbar(scatter,label="Label by k-means")
  plt.title("t-SNE + k-means of 2D Ising configuration")
  plt.xlabel("t-SNE 1")
  plt.ylabel("t-SNE 2")
  plt.grid(True)
  plt.show()

In [ ]:
visualize_tsne2_kmeans2(xs_tsne_,cluster_labels_)

# Autoencoder
Use the autoencoder to illustrate the structure in the two-dimensional space.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
xs_uni_=(xs_vct_+1)/2

In [ ]:
dim_latent_=2
dim_input_=xs_uni_.shape[1]

In [ ]:
encoder=models.Sequential([
    layers.Input(shape=(dim_input_,)),
    layers.Dense(dim_input_//2,activation="relu"),
    layers.Dense(dim_latent_)
])

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(encoder,
           to_file="encoder.png",
           show_shapes=True,
           show_layer_names=True,
           dpi=100)

Image(filename="encoder.png", width=400)

In [ ]:
decoder=models.Sequential([
    layers.Input(shape=(dim_latent_,)),
    layers.Dense(dim_input_//2,activation="relu"),
    layers.Dense(dim_input_,activation="sigmoid")
])

In [ ]:
plot_model(decoder,
           to_file="decoder.png",
           show_shapes=True,
           show_layer_names=True,
           dpi=100)

Image(filename="decoder.png", width=400)

In [ ]:
autoencoder=models.Sequential([
    encoder,
    decoder
])

In [ ]:
plot_model(autoencoder,
           to_file="autoencoder.png",
           show_shapes=True,
           show_layer_names=True,
           dpi=100)

Image(filename="autoencoder.png", width=400)

In [ ]:
autoencoder.compile(optimizer="adam", loss="mse")

In [ ]:
autoencoder.fit(xs_uni_,xs_uni_, epochs=20,batch_size=64,validation_split=0.1)

In [ ]:
zs_=encoder.predict(xs_uni_)

In [ ]:
def visualize_autoencoder2(data,label):
  plt.figure(figsize=(16,9))
  scatter=plt.scatter(data[:,0],data[:,1],c=label,cmap="coolwarm",s=10)
  plt.colorbar(scatter,label="True label (1: ordered; 0: disordered)")
  plt.title("Autoencoder")
  plt.xlabel("latent variable 1")
  plt.ylabel("latent variable 2")
  plt.grid(True)
  plt.show()

In [ ]:
visualize_autoencoder2(zs_,ys_)

# Consider the spin flip symmetry

## Trial 1
Extend the dataset to include the flipped configuration.

In [ ]:
xs_flip_ = -xs_
ys_flip_ = ys_.copy()

In [ ]:
xs_extended_=np.concatenate([xs_,xs_flip_],axis=0)
ys_extended_=np.concatenate([ys_,ys_flip_],axis=0)

In [ ]:
xs_extended_vct_=xs_extended_.reshape(len(xs_extended_),-1).astype("float64")
xs_extended_uni_=(xs_extended_vct_+1)/2

In [ ]:
autoencoder.fit(xs_extended_uni_,xs_extended_uni_, epochs=20,batch_size=64,validation_split=0.1)

In [ ]:
zs_extended_=encoder.predict(xs_extended_uni_)

In [ ]:
visualize_autoencoder2(zs_extended_,ys_extended_)

## Trial 2
Introduce the triplet loss function.

In [ ]:
def create_triplet_flipsym(data):
  N=len(data)
  triplets=[]

  for i in range(N):
    anchor=data[i]
    positive=-anchor # flipped configuration is in the same phase
    j = np.random.choice([k for k in range(N) if k!=i])
    negative=data[j]

    triplets.append((anchor,positive,negative))

  return triplets

In [ ]:
triplets_=create_triplet_flipsym(xs_)

In [ ]:
as_=np.array([t[0] for t in triplets_])
as_ = as_.reshape(len(as_),-1).astype("float64")
as_=(as_+1)/2

In [ ]:
ps_=np.array([t[0] for t in triplets_])
ps_ = ps_.reshape(len(ps_),-1).astype("float64")
ps_=(ps_+1)/2

In [ ]:
ns_=np.array([t[0] for t in triplets_])
ns_ = ns_.reshape(len(ns_),-1).astype("float64")
ns_=(ns_+1)/2

In [ ]:
from tensorflow.keras import Input

In [ ]:
dim_latent_=2
dim_input_=as_.shape[1]

In [ ]:
encoder=models.Sequential([
      layers.Input(shape=(dim_input_,)),
      layers.Dense(dim_input_//2, activation="relu"),
      layers.Dense(dim_latent_)
  ])

In [ ]:
plot_model(encoder,
           to_file="encoder.png",
           show_shapes=True,
           show_layer_names=True,
           dpi=100)

Image(filename="encoder.png", width=400)

In [ ]:
in_as_=Input(shape=(dim_input_,))
in_ps_=Input(shape=(dim_input_,))
in_ns_=Input(shape=(dim_input_,))

In [ ]:
out_as_=encoder(in_as_)
out_ps_=encoder(in_ps_)
out_ns_=encoder(in_ns_)

In [ ]:
triplet_model=models.Model(
    inputs=[in_as_,in_ps_,in_ns_],
    outputs=layers.Concatenate()([out_as_,out_ps_,out_ns_])
)

In [ ]:
plot_model(triplet_model,
           to_file="triplet_model.png",
           show_shapes=True,
           show_layer_names=True,
           dpi=100)

Image(filename="triplet_model.png", width=600)

In [ ]:
def triplet_loss(margin=1.0):
  """
  triplet loss function
  """
  def loss(y_true, y_pred):
    z_a=y_pred[:,:dim_latent_]
    z_p=y_pred[:,dim_latent_:2*dim_latent_]
    z_n=y_pred[:,2*dim_latent_:]

    d_ap=tf.reduce_sum(tf.square(z_a-z_p), axis=1)
    d_an=tf.reduce_sum(tf.square(z_a-z_n), axis=1)

    loss=tf.maximum(d_ap-d_an+margin,0.0)
    return tf.reduce_mean(loss)
  return loss

In [ ]:
triplet_model.compile(optimizer="adam",loss=triplet_loss(margin=1.0))

In [ ]:
triplet_model.fit(
    [as_,ps_,ns_],
    np.zeros(len(as_)), # dummy taraining labels
    batch_size=64, epochs=40
)

In [ ]:
zs_tri_=encoder.predict(xs_uni_)

In [ ]:
def visualize_triplet_loss2(data,label):
  plt.figure(figsize=(16,9))
  scatter=plt.scatter(data[:,0],data[:,1],c=label,cmap="coolwarm",s=10)
  plt.colorbar(scatter,label="True label (1: ordered; 0: disordered)")
  plt.title("Latent space (triplet loss: weak supervised)")
  plt.xlabel("latent variable 1")
  plt.ylabel("latent variable 2")
  plt.grid(True)
  plt.show()

In [ ]:
visualize_triplet_loss2(zs_tri_,ys_)

In [ ]:
zs_tri_tsne_ = TSNE(n_components=2, perplexity=30).fit_transform(zs_tri_)

In [ ]:
def visualize_triplet_loss_tsne2(data,label):
  plt.figure(figsize=(16,9))
  scatter=plt.scatter(data[:,0],data[:,1],c=label,cmap="coolwarm",s=10)
  plt.colorbar(scatter,label="True label (1: ordered; 0: disordered)")
  plt.title("t-SNE of Triplet Loss latent space")
  plt.xlabel("latent variable 1")
  plt.ylabel("latent variable 2")
  plt.grid(True)
  plt.show()

In [ ]:
visualize_triplet_loss_tsne2(zs_tri_tsne_,ys_)